In [1]:
from typing import Tuple

import itertools
import logging

import numpy
import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

In [2]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('prophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('fbprophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [4]:
country = 'canada'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [5]:
def make_dataset(df_processed: pandas.DataFrame) -> pandas.DataFrame:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)

    # unemployment rate, being a percentage, can only be between 0 to 100
    df_mrd['floor'] = 0
    df_mrd['cap'] = 100
    return df_mrd

In [6]:
df_mrd = make_dataset(df)

In [7]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [8]:
df_train, df_test = train_test_split(df_mrd, 12)

In [9]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse'):
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # other params
    model_params = {
        'growth': 'logistic', 'seasonality_mode': 'additive',
        'daily_seasonality': False, 'weekly_seasonality': False, 'yearly_seasonality': False
    }
    cv_params = {'horizon': '30 days', 'parallel': 'processes'}
    pm_params = {'metrics': [eval_metric, 'coverage'], 'rolling_window': 0.1, 'monthly': True}

    print(f'Total hyper-parameter set count: {len(all_params)}')

    # Use cross validation to evaluate all parameters
    iteration_count: int = 1
    for hyper_params in all_params:
        print(f'Set: {iteration_count}')
        print(f'{hyper_params}')

        m = Prophet(**hyper_params, **model_params).add_seasonality(
            name='quarterly', period=365.25/4, fourier_order = 5
            ).fit(df)
        df_cv = cross_validation(m, **cv_params)
        df_p = performance_metrics(df_cv, **pm_params)
        eval_metric_values.append(df_p[eval_metric].values[0])

        print(f'{eval_metric.upper()}: {df_p[eval_metric].values[0]}')
        iteration_count += 1
        print()

    best_params: dict = all_params[numpy.argmin(eval_metric_values)]

    print('Training model on the best hyper-parameter set.')
    print(f'{best_params}')
    
    auto_model: Prophet = Prophet(**best_params, **model_params).add_seasonality(
        name='quarterly', period=365.25/4, fourier_order = 5
        ).fit(df)

    print('Cross-Validating best model.')
    auto_model_cv = cross_validation(auto_model, **cv_params)
    auto_model_p: pandas.DataFrame = performance_metrics(auto_model_cv, **pm_params)

    return auto_model, auto_model_p, best_params

In [10]:
param_grid = {  
    'changepoint_prior_scale': [0.5, 0.75],
    'seasonality_prior_scale': [0.1, 1.0],
}

auto_model, auto_model_p, best_params = train_model(df_train, param_grid)

Total hyper-parameter set count: 4
Set: 1
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}
RMSE: 1.2641109217686224

Set: 2
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 1.2689332754511848

Set: 3
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 0.1}
RMSE: 1.2572105847999606

Set: 4
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 1.0}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 1.2636311957996869

Training model on the best hyper-parameter set.
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 0.1}
Cross-Validating best model.


In [11]:
best_params

{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 0.1}

In [12]:
auto_model_p

,horizon,rmse,coverage
0,1,1.257211,0.346105
1,2,1.262396,0.328469


In [13]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [14]:
df_predicted, rmse = test_model(df_test, auto_model)

In [15]:
df_predicted

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,quarterly,quarterly_lower,quarterly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-01,17.055869,100,0,5.982970,8.111000,17.055869,17.055869,-10.017581,-10.017581,-10.017581,-10.017581,-10.017581,-10.017581,0.0,0.0,0.0,7.038288
1,2022-05-01,17.054628,100,0,6.190796,8.146837,17.054553,17.054553,-9.872643,-9.872643,-9.872643,-9.872643,-9.872643,-9.872643,0.0,0.0,0.0,7.181985
2,2022-06-01,17.053347,100,0,5.881366,8.031058,17.053178,17.053178,-10.102599,-10.102599,-10.102599,-10.102599,-10.102599,-10.102599,0.0,0.0,0.0,6.950747
3,2022-07-01,17.052106,100,0,6.135369,8.141119,17.051804,17.051804,-9.930748,-9.930748,-9.930748,-9.930748,-9.930748,-9.930748,0.0,0.0,0.0,7.121358
4,2022-08-01,17.050825,100,0,5.931180,7.973755,17.050092,17.050092,-10.069830,-10.069830,-10.069830,-10.069830,-10.069830,-10.069830,0.0,0.0,0.0,6.980995
5,2022-09-01,17.049543,100,0,6.042236,8.033527,17.048209,17.048209,-10.002329,-10.002329,-10.002329,-10.002329,-10.002329,-10.002329,0.0,0.0,0.0,7.047214
6,2022-10-01,17.048303,100,0,5.893168,7.953943,17.046376,17.046376,-10.064924,-10.064924,-10.064924,-10.064924,-10.064924,-10.064924,0.0,0.0,0.0,6.983379
7,2022-11-01,17.047022,100,0,5.935990,7.989226,17.044195,17.044195,-10.110379,-10.110379,-10.110379,-10.110379,-10.110379,-10.110379,0.0,0.0,0.0,6.936643
8,2022-12-01,17.045782,100,0,5.916425,7.975050,17.039202,17.048735,-10.077629,-10.077629,-10.077629,-10.077629,-10.077629,-10.077629,0.0,0.0,0.0,6.968153
9,2023-01-01,17.044501,100,0,6.090516,8.069809,17.018132,17.067480,-9.988086,-9.988086,-9.988086,-9.988086,-9.988086,-9.988086,0.0,0.0,0.0,7.056415


In [16]:
rmse

1.95774436212555

In [17]:
with open(f'{country}_prophet_cv_model_2.json', 'w') as f:
    f.write(model_to_json(auto_model))

In [20]:
df_future = pandas.DataFrame(data = {
        'ds': ['2023-03-01', '2023-04-01', '2023-05-01'],
        'floor': [0, 0, 0],
        'cap': [100, 100, 100],
    }
)

In [21]:
df_future_prediction = auto_model.predict(df_future)

In [22]:
df_future_prediction

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,quarterly,quarterly_lower,quarterly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,17.042063,100,0,6.198168,8.148776,17.042063,17.042063,-9.849816,-9.849816,-9.849816,-9.849816,-9.849816,-9.849816,0.0,0.0,0.0,7.192247
1,2023-04-01,17.040782,100,0,6.057139,8.056827,17.040575,17.040575,-9.951525,-9.951525,-9.951525,-9.951525,-9.951525,-9.951525,0.0,0.0,0.0,7.089257
2,2023-05-01,17.039542,100,0,6.197609,8.324749,17.039100,17.039100,-9.762600,-9.762600,-9.762600,-9.762600,-9.762600,-9.762600,0.0,0.0,0.0,7.276942
